# FIAP - Atividade F3A4 - Grupo 61

> Esse projeto é parte do curso de **Inteligência Artificial** da [FIAP](https://github.com/fiap) - Online 2024. Este notebook é a atividade "**Fase 3** Atividade Cap. 14 - A primeira técnica de aprendizado de máquina."

## Objetivos

1. Desenvolver análise exploratória dos dados
2. Desenvolver análise descritiva dos dados
3. Responder ao seguinte questionamento: `Encontrar o “perfil ideal” de solo/clima para as plantações, além de discorrer sobre como os três produtos distintos (à escolha do grupo) se comparam com esse perfil ideal. Por exemplo, preferem maior umidade e mais precipitação? Preferem mais calor e menos fósforo? Para esta parte se apoie em análises estatísticas e/ou visuais;`
4. Desenvolver modelos preditivos de classificação para os dados

## Integrantes

- Bruno Conterato (RM561048)
- Luis Emidio (RM559976)
- Willian Pinheiro Marques (RM560402)
- Roberto Besser (RM559400)
- Ludimila Vitorino (RM559697)

## Professores

- Tutor: <a href="https://www.linkedin.com/in/lucas-gomes-moreira-15a8452a/">Lucas Gomes Moreira</a>
- Coordenador: <a href="https://www.linkedin.com/in/profandregodoi/">André Godoi</a>

## 1. Definição do Problema

## 2. Preparação do Ambiente

In [ ]:
# Import necessary libraries
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from scipy.stats import zscore
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Set the dark theme for plots
plt.style.use('dark_background')

## 3. Coleta de Dados

In [ ]:
# Load dataset

df = pd.read_csv('./data/dataset.csv')
df.sample(5)


## 1. Análise Exploratória de Dados

### 1.2. Compreensão dos dados

#### 1.2.1. Visualização em tabela dos dados

In [ ]:
print("Quantidade de linhas: ", df.shape[0])
print("Quantidade de colunas: ", df.shape[1])

# Display 5 random samples from the dataset
df.sample(5)

#### 1.2.2. Entendendo os dados

**Variáveis numéricas discretas**
- N: concentração de potássio (unidade não informada)
- P: concentração de fósforo (unidade não informada)
- K: concentração de potássio (unidade não informada)

**Variáveis numéricas contínuas**
- temperature: temperatura (ºC)
- humidity: umidade (%)
- ph: H do solo
- rainfall: quantidade de precipitação (mm)

**Variaveis categóricas**
- label: cultura agrícola


In [ ]:
df.info()

#### 1.2.3. Visualizando as classes (Culturas agrícolas) 

- Culturas agrícolas disponíveis:
  - rice (arroz)
  - maize (milho)
  - chickpea (grão-de-bico)
  - kidneybeans (feijão)
  - pigeonpeas (ervilhas)
  - mothbeans (feijão-moth)
  - mungbean (feijão-mungo)
  - blackgram (feijão-preto)
  - lentil (lentilha)
  - pomegranate (romã)
  - banana (banana)
  - mango (manga)
  - grapes (uvas)
  - watermelon (melancia)
  - muskmelon (melão)
  - apple (maçã)
  - orange (laranja)
  - papaya (papaia)
  - coconut (coco)
  - cotton (algodão)
  - jute (juta)
  - coffee (café)
- Total de culturas agrícolas: 22
- Cada cultura agrícola possui 100 registros
- Total de registros: 2200

In [ ]:
# Contagem dos valores para cada classe
classes = df['label'].value_counts()
classes


#### 1.2.4. Análise de Inconsistências

**Investigação de outliers**

- Cada cultura agrícola é cultivada em diferentes condições de solo, temperatura, umidade, pH e precipitação
- Portanto, não faz sentido considerar outliers nos dados entre culturas diferentes
- A análise de outliers deve, portanto, ser feita para cada cultura agrícola individualmente

**Método de detecção de outliers**

Vamos utilizar o método do z_score para detectar outliers.

Cálculo do z_score:

`z = (X - μ) / σ`
- X é o valor da amostra
- μ é a média da população
- σ é o desvio padrão da população

O z_score é uma medida de quantos desvios padrão um ponto de dados está longe da média. Se o z_score de um ponto de dados for maior que 2, consideramos o ponto como um outlier.

Consideremos a população como sendo dados de cada coluna para cada cultura agrícola.

**Outlivers encontrados**
- Apenas 2 pontos outliers encontrados: para a cultura agrícola 'apple' (maçã) na variável 'ph'
  
**Conclusão**
- Os dados não contém quantidade significativa de outliers, sendo assim não foi necessário a correção de inconsistências.
- Remoção de duplicatas são não-aplicáveis

In [ ]:
feature_columns = df.select_dtypes(include=[float, int]).columns
for crop in df['label'].unique():
    print(f"Outliers para a classe {crop}")
    z_scores = df[df['label'] == crop][feature_columns].apply(zscore)
    outliers = (z_scores.abs() > 2).sum()
    
    if len(outliers[outliers > 0]) == 0:
        print("Nenhum outlier encontrado")
        print("\n")
        continue
    
    print(outliers[outliers > 0])
    print("\n")


In [ ]:
# Filter z-scores for 'apple' crop
apple_z_scores = df[df['label'] == 'apple'][feature_columns].apply(zscore)

# Find outliers in 'ph' variable
apple_ph_outliers = apple_z_scores[apple_z_scores['ph'].abs() > 2]
# Print the top 10 absolute z-scores for 'apple' crop
top_10_z_scores_indices = apple_z_scores['ph'].abs().sort_values(ascending=False).head(10).index

print("Top 10 absolute z-scores for 'apple' crop with their 'ph' values:")
for idx in top_10_z_scores_indices:
    print(f"Index: {idx}, z_score: {apple_z_scores.loc[idx, 'ph']}, ph value: {df.loc[idx, 'ph']}")

#### 1.2.5. Buscando correlações entre as variáveis

- Em termos gerais, as correlações entre as variáveis são baixas (valores absolutos menores que 23.14%)
- A excessão: a correlação entre as variáveis N e K é de 73.62%, considerada elevada.


In [ ]:
# Correlation table
corr_df = df.corr(numeric_only=True)
corr_df

#### 1.2.6. Estatísticas descritivas básicas das variáveis numéricas

In [ ]:
# Summary statistics of the dataset
df.describe()

#### 1.2.7. Análise de valores nulos

Não há valores nulos nos dados

In [ ]:
# Contar valores nulos por coluna
df.isnull().sum()

## 2. Análise descritiva

### 2.1. Gráficos boxplot de cada variável por cultura agrícola

#### 2.1.1. Boxplot Nitrogênio (N) por cultura agrícola

Por meio desta visualização conseguimos avaliar o intervalo de concentração de nitrogênio (N) ideal para cada cultura agrícola. 

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='label', y='N', palette='Set2', legend=False)
plt.title('Nitrogen Levels by Crop Type')
plt.xlabel('Crop Type')
plt.ylabel('Nitrogen (N) Concentration')
plt.xticks(rotation=45)
plt.grid()
plt.show()

#### 2.1.2. Boxplot Fósforo (P) por cultura agrícola

Por meio desta visualização conseguimos avaliar o intervalo de concentração de fósforo (P) ideal para cada cultura agrícola. 

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='label', y='P', palette='Set2')
plt.title('Phosphorus Levels by Crop Type')
plt.xlabel('Crop Type')
plt.ylabel('Phosphorus (P) Concentration')
plt.xticks(rotation=45)
plt.grid()
plt.show()

#### 2.1.3. Boxplot Potássio (K) por cultura agrícola

Por meio desta visualização conseguimos avaliar o intervalo de concentração de potássio (K) ideal para cada cultura agrícola. 

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='label', y='K', palette='Set2')
plt.title('Potassium Levels by Crop Type')
plt.xlabel('Crop Type')
plt.ylabel('Potassium (K) Concentration')
plt.xticks(rotation=45)
plt.grid()
plt.show()

#### 2.1.4. Boxplot Temperatura por cultura agrícola

Por meio desta visualização conseguimos avaliar o intervalo de temperatura ideal para cada cultura agrícola. 

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='label', y='temperature', palette='Set2')
plt.title('Temperature by Crop Type')
plt.xlabel('Crop Type')
plt.ylabel('Temperature')
plt.xticks(rotation=45)
plt.grid()
plt.show()

#### 2.1.5. Boxplot Umidade (%) por cultura agrícola

Por meio desta visualização conseguimos avaliar o intervalo de umidade (%) ideal para cada cultura agrícola. 

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='label', y='humidity', palette='Set2')
plt.title('Humidity by Crop Type')
plt.xlabel('Crop Type')
plt.ylabel('Humidity (%)')
plt.xticks(rotation=45)
plt.grid()
plt.show()

#### 2.1.6. Boxplot pH por cultura agrícola

Por meio desta visualização conseguimos avaliar o intervalo de pH ideal para cada cultura agrícola. 

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='label', y='ph', palette='Set2')
plt.title('pH by Crop Type')
plt.xlabel('Crop Type')
plt.ylabel('pH')
plt.xticks(rotation=45)
plt.grid()
plt.show()

#### 2.1.7. Boxplot quantidade de precipitação por cultura agrícola

Por meio desta visualização conseguimos avaliar o intervalo de quantidade de precipitação ideal para cada cultura agrícola. 

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='label', y='rainfall', palette='Set2')
plt.title('Rainfall by Crop Type')
plt.xlabel('Crop Type')
plt.ylabel('Rainfall (mm)')
plt.xticks(rotation=45)
plt.grid()
plt.show()

### 2.2. Gráficos de barras para média de temperatura e umidade por cultura agrícola

#### 2.2.1. Média de temperatura por cultura agrícola

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='label', y='temperature', palette='Set2')
plt.title('Temperature by Crop Type')
plt.xlabel('Crop Type')
plt.ylabel('Temperature')
plt.xticks(rotation=45)
plt.grid()
plt.show()

#### 2.2.2. Média de umidade por cultura agrícola

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='label', y='humidity', palette='Set2', estimator=sum)
plt.title('Average Humidity by Crop Type')
plt.xticks(rotation=45)
plt.xlabel('Crop Type')
plt.ylabel('Humidity (%)')
plt.grid()
plt.show()

#### 2.2.3. Média de precipitação por cultura agrícola

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='label', y='rainfall', palette='Set2', estimator=sum)
plt.title('Average Rainfall by Crop Type')
plt.xticks(rotation=45)
plt.xlabel('Crop Type')
plt.ylabel('Rainfall (mm)')
plt.grid()
plt.show()

### 2.3. Gráficos de barras para média de quantidade de precipitação por cultura agrícola

A linha ciano representa a estimativa da Função de Densidade de Probabilidade (PDF) da quantidade de precipitação para cada cultura agrícola.

In [ ]:
# 2.3. Rainfall distribution
plt.figure(figsize=(10, 6))
sns.histplot(df['rainfall'], bins=20, kde=True, color='cyan')
plt.title('Rainfall Distribution')
plt.xlabel('Rainfall (mm)')
plt.show()

### 2.4. Matriz de correlação entre as variáveis numéricas

Escala de cores baseada em calor:
- Azul indica menor correlação (em valores absolutos)
- Vermelho indica maior a correlação (em valores absolutos)

In [ ]:
# Correlação entre as features numéricas
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
plt.figure(figsize=(12, 8))
sns.heatmap(df.select_dtypes(include=numerics).corr(), annot=True, cmap='coolwarm')
plt.title("Matriz de Correlação")
plt.show()

### 2.5. Gráficos de pontos de temperatura por umidade para cada cultura agrícola

Objetivos:
- Avaliar os intervalos ideais de temperatura e umidade para cada cultura agrícola.
- Responder à pergunta: `Encontrar o “perfil ideal” de solo/clima para as plantações`

A fim de cumprir o objetivo, esta visualização mostra um gráfico para cada cultura e fixa os intevalos dos eixos x e y.
- Eixo x: temperatura fixada entre 0 e 40 ºC
- Eixo y: umidade fixada entre 0 e 100 %

Desta forma, conseguimos visualizar a distribuição dos dados de temperatura e umidade para cada cultura agrícola.

In [ ]:
culturas = df['label'].unique()

# Definir limites fixos com base nos dados, adicionando uma margem
temp_min_fixed = 00  # Um pouco abaixo do mínimo
temp_max_fixed = 45  # Um pouco acima do máximo
hum_min_fixed = 0   # Um pouco abaixo do mínimo
hum_max_fixed = 100   # Um pouco acima do máximo

# Loop para criar um gráfico para cada cultura
for cultura in culturas:
    # Filtrar os dados para a cultura atual
    data_cultura = df[df['label'] == cultura]
    
    # Criar o gráfico de dispersão
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x='temperature', y='humidity', data=data_cultura)
    plt.title(f'Temperatura vs. Umidade para {cultura.capitalize()}')
    plt.xlabel('Temperatura (°C)')
    plt.ylabel('Umidade (%)')
    
    # Definir os limites dos eixos
    plt.xlim(temp_min_fixed, temp_max_fixed)
    plt.ylim(hum_min_fixed, hum_max_fixed)
    
    plt.show()

# Criar o gráfico de dispersão
plt.figure(figsize=(10, 6))
sns.scatterplot(x='temperature', y='humidity', hue='label',data=df)
plt.title(f'Temperatura vs. Umidade')
plt.xlabel('Temperatura (°C)')
plt.ylabel('Umidade (%)')
plt.show()

### 2.6. Distribuição de valores de pH (empilhada) por cultura agrícola

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='ph', hue='label', multiple='stack', bins=20)
plt.title('Distribuição do pH do Solo por Cultura')
plt.xlabel('pH do Solo')
plt.ylabel('Frequência')
plt.show()

### 2.7. Gráficos de correlação bi-variadas para cada cultura agrícola

Objetivo:
- Avaliar a relação entre as variáveis para cada cultura agrícola

Obs.: vamos fazer apenas para a cultura agrícola 'coffee' (café) para demonstração.

In [ ]:
# Correlações bi-variadas para uma cultura específica
_ = sns.pairplot(df[df["label"] == "coffee"])

In [ ]:
# Gráfico de dispersão 3D para N, P e K
fig = plt.figure(figsize=(20, 14))
ax = fig.add_subplot(111, projection='3d')

# Converter labels para números
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])
labels = le.classes_

scatter = ax.scatter(df['N'], df['P'], df['K'], c=df['label_encoded'], cmap='viridis')
ax.set_title('N vs. P vs. K por Cultura')
ax.set_xlabel('Nitrogênio (N)')
ax.set_ylabel('Fósforo (P)')
ax.set_zlabel('Potássio (K)')

# Legenda personalizada
legend = ax.legend(handles=scatter.legend_elements()[0], labels=labels)
ax.add_artist(legend)

plt.show()

### 2.8 Principais achados da base

## 3. Perfil Ideal

## 4. Machine Learning

Sumário:
1. Pré-processamento de Dados: limpeza, transformação e feature engineering
2. Dividir dos Dados
3. Algoritmos
   1. Treinamento
   2. Avaliação
4. Concluir

### 4.1 Definição do Problema

Objetivos:
- Desenvolver modelos preditivos de classificação de cultura agrícola baseando-se nos dados

### 4.3 Pré-processamento de Dados

O pré-processamento de dados é essencial para garantir que o modelo de Machine Learning receba dados de alta qualidade. Ele abrange desde a limpeza e transformação dos dados até a criação de novas features, passando pela identificação e tratamento de outliers. Cada uma dessas etapas contribui para a melhoria da precisão e eficácia do modelo.


#### 4.3.1 Limpeza de Dados

Tratamento de dados ausentes, duplicados e inconsistências.

- Não aplicável, pois não há valores ausentes, duplicatas ou inconsistências.

#### 4.4.3 Feature Engineering

##### 4.4.3.1 Criação de Novas Features

- Não aplicável

##### 4.4.3.2 Transformação de Variáveis Categóricas

In [ ]:
# Gerar label-encoding para a variável categórica 'label'

df_label = df['label']

le = LabelEncoder()
df_label = le.fit_transform(df_label)

print(df_label)


In [ ]:
df_features = df.drop(columns=['label', 'label_encoded'], errors='ignore')
df_features

#### 4.3.2 Divisão dos dados

In [ ]:
# 30% dos dados para teste
df_train, df_test, df_train_label, df_test_label = train_test_split(df_features, df_label, test_size=0.4, random_state=42)

print("Quantidade de linhas no dataset de treino: ", df_train.shape[0])
print("Quantidade de linhas no dataset de teste: ", df_test.shape[0])


#### 4.3.3 Transformação de Dados

##### 4.3.3.1 Normalização

Isso é útil para algoritmos sensíveis à escala, como redes neurais.

A normalização dos dados pode melhorar o desempenho de alguns algoritmos de aprendizado de máquina categóricos, mas não de todos. Aqui está uma análise de como a normalização afeta cada um dos algoritmos:

Algoritmos afetados pela normalização:

- **KNN (k-Nearest Neighbors)**: KNN usa a distância euclidiana para calcular a similaridade entre os pontos de dados. Quando as variáveis ​​têm escalas diferentes, as variáveis ​​com valores maiores podem ter um impacto desproporcional na distância. A normalização garante que todas as variáveis ​​tenham o mesmo peso na medida da distância, o que pode melhorar a precisão do KNN.
- **Regressão Logística**: A regressão logística usa a função sigmóide, que é sensível à escala das variáveis ​​de entrada. Se algumas variáveis ​​tiverem valores muito maiores do que outras, elas podem dominar a função sigmóide, levando a uma convergência lenta ou mesmo a um desempenho ruim. A normalização pode ajudar a resolver esse problema, garantindo que todas as variáveis ​​tenham uma escala semelhante.
- **SVM (Máquina de Vetores de Suporte)**: SVM também usa a distância euclidiana para encontrar o hiperplano ótimo. Da mesma forma com o KNN, variáveis ​​com escalas diferentes podem influenciar a distância de forma desigual. A normalização pode ajudar a melhorar o desempenho do SVM, especialmente em conjuntos de dados com características de diferentes escalas.

Algoritmos menos afetados pela normalização:

- **Árvores de Decisão**: Árvores de decisão são menos sensíveis à escala dos dados, pois a divisão de nós é baseada na seleção da melhor variável e ponto de corte, independentemente de sua escala.
- **Florestas Aleatórias**: Como as árvores de decisão, as florestas aleatórias são relativamente insensíveis à normalização, pois são compostas por várias árvores de decisão, que são treinadas em diferentes subconjuntos de dados.

In [ ]:
scaler = MinMaxScaler().fit(df_train)
df_train_scaled = scaler.transform(df_train)
df_test_scaled = scaler.transform(df_test)

# Verificar os dados normalizados
df_train_scaled

##### 4.3.3.2 Padronização

Importante para algoritmos sensíveis à escala.

Aplicável?

### 4.4. Algoritmos

#### 4.4.1. Árvore de Decisão

Uma técnica de aprendizado supervisionado que cria uma estrutura hierárquica para classificação e regressão, baseada em decisões sequenciais.

**Funcionamento:**
- **Seleção da melhor divisão**: Identifica a variável que melhor divide os dados em subconjuntos homogêneos
- **Divisão recursiva**: Divide os dados em subconjuntos, formando novos nós
- **Classificação**: Classifica novos exemplos percorrendo a árvore da raiz até as folhas

**Vantagens:**
- Fácil interpretação
- Flexibilidade para diferentes tipos de dados
- Lida bem com variáveis numéricas e categóricas

**Desvantagens:**
- Tendência a overfitting
- Instabilidade com pequenas variações nos dados
- Viés para variáveis com mais categorias

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(df_train_scaled, df_train_label)

# Prever as classes para o conjunto de teste
dt_pred = dt.predict(df_test_scaled)

# Calcular a acurácia
accuracy = accuracy_score(df_test_label, dt_pred)
print(f'Acurácia do modelo Decision Tree: {accuracy:.2f}')

# Gerar um relatório de classificação
print(classification_report(df_test_label, dt_pred))

#### 4.4.2. Floresta Aleatória

Uma evolução da árvore de decisão que combina múltiplas árvores para melhorar a capacidade preditiva.

**Funcionamento:**
- **Criação de múltiplas árvores**: Treina várias árvores com subconjuntos aleatórios dos dados
- **Agregação**: Combina as previsões de todas as árvores por votação ou média

**Vantagens:**
- Alta precisão
- Robustez contra overfitting
- Boa performance com dados de alta dimensionalidade

**Desvantagens:**
- Menor interpretabilidade
- Alto custo computacional

In [ ]:
rf = RandomForestClassifier(n_estimators=25)
rf.fit(df_train_scaled, df_train_label)

rf_pred = rf.predict(df_test_scaled)

print("Acurácia do modelo Random Forest: ", accuracy_score(df_test_label, rf_pred))
print(classification_report(df_test_label, rf_pred))


#### 4.4.3. Regressão Logística

Algoritmo de aprendizado supervisionado que pode ser adaptado para classificação multiclasse através da abordagem "one-vs-all" ou "one-vs-one".

**Funcionamento:**
- **Combinação linear**: Combina características com seus pesos para cada classe
- **Função logística**: Aplica função softmax para obter probabilidades para todas as classes
- **Classificação**: Atribui o exemplo à classe com maior probabilidade prevista

**Vantagens:**
- Alta interpretabilidade das probabilidades por classe
- Eficiente para grandes datasets
- Robusta com regularização
- Fornece probabilidades bem calibradas
- Fácil de atualizar com novos dados

**Desvantagens:**
- Performance pode ser inferior a outros métodos em relações não-lineares complexas
- Sensível à multicolinearidade entre variáveis preditoras
- Pode requerer mais recursos computacionais conforme aumenta o número de classes
- Necessita de uma quantidade significativa de dados por classe
- Assume independência entre as classes

In [ ]:
lr = LogisticRegression(max_iter=1000)
lr.fit(df_train_scaled, df_train_label)

lr_pred = lr.predict(df_test_scaled)

print("Acurácia do modelo Logistic Regression: ", accuracy_score(df_test_label, lr_pred))
print(classification_report(df_test_label, lr_pred))


#### 4.4.4 K-Nearest Neighbors (KNN)

Algoritmo baseado na proximidade entre pontos para classificação.

**Funcionamento:**
- **Cálculo de distância**: Mede a distância entre o novo ponto e todos os pontos existentes
- **Seleção de vizinhos**: Identifica os K vizinhos mais próximos
- **Votação**: Classifica baseado na maioria dos vizinhos

**Vantagens:**
- Simples de implementar
- Não requer treinamento
- Adaptável a novos dados

**Desvantagens:**
- Computacionalmente intensivo para grandes datasets
- Sensível à escolha do valor K
- Requer normalização dos dados


In [ ]:
knn = KNeighborsClassifier(n_neighbors=9)
knn.fit(df_train_scaled, df_train_label)

knn_pred = knn.predict(df_test_scaled)

print("Acurácia do modelo KNN: ", accuracy_score(df_test_label, knn_pred))
print(classification_report(df_test_label, knn_pred))


#### 4.4.5 Support Vector Machine (SVM)

Algoritmo que busca o melhor hiperplano de separação entre classes.

**Funcionamento:**
- **Mapeamento dimensional**: Transforma dados para dimensões superiores
- **Otimização**: Encontra o hiperplano com maior margem entre classes
- **Classificação**: Determina classes pela posição relativa ao hiperplano

**Vantagens:**
- Eficaz em alta dimensionalidade
- Robusto a outliers
- Versátil com diferentes kernels

**Desvantagens:**
- Complexidade na escolha de parâmetros
- Alto custo computacional para grandes datasets

In [ ]:
svm = SVC()
svm.fit(df_train_scaled, df_train_label)

svm_pred = svm.predict(df_test_scaled)

print("Acurácia do modelo SVM: ", accuracy_score(df_test_label, svm_pred))
print(classification_report(df_test_label, svm_pred))


### 4.5 Model Regularization

In [147]:
# Import necessary libraries
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv('./data/dataset.csv')
print(df.head())

# Split the data
df_train, df_test = train_test_split(df, test_size=0.6, random_state=42)

# Then perform label encoding on the training set
le = LabelEncoder()
df_train['label_encoded'] = le.fit_transform(df_train['label'])

# Apply the same encoder to the test set without fitting
df_test['label_encoded'] = le.transform(df_test['label'])

# Normalize the data using MinMaxScaler
scaler = MinMaxScaler()
df_train_scaled = scaler.fit_transform(df_train.drop(columns=['label', 'label_encoded']))
df_test_scaled = scaler.transform(df_test.drop(columns=['label', 'label_encoded']))

print(df_train_scaled)
print("\n")
print(df_test_scaled)

# Function to perform cross-validation and display results
def evaluate_model(model, model_name):
    model.fit(df_train_scaled, df_train['label_encoded'])

    # Cross-validation accuracy
    cv_scores = cross_val_score(model, df_train_scaled, df_train['label_encoded'], cv=5)
    print(f'Cross-validated accuracy for {model_name}: {np.mean(cv_scores):.2f} ± {np.std(cv_scores):.2f}')

    # Test set accuracy
    predictions = model.predict(df_test_scaled)
    print(f'Test set accuracy for {model_name}: {accuracy_score(df_test['label_encoded'], predictions):.2f}')
    print(classification_report(df_test['label_encoded'], predictions))

# Decision Tree Classifier with max depth to reduce overfitting
dt = DecisionTreeClassifier(max_depth=5)
evaluate_model(dt, "Decision Tree")

# Random Forest Classifier with limited depth
rf = RandomForestClassifier(n_estimators=25, max_depth=10)
evaluate_model(rf, "Random Forest")

# Logistic Regression
lr = LogisticRegression(max_iter=1000)
evaluate_model(lr, "Logistic Regression")

# K-Nearest Neighbors (KNN)
knn = KNeighborsClassifier(n_neighbors=9)
evaluate_model(knn, "KNN")

# Support Vector Machine (SVM)
svm = SVC()
evaluate_model(svm, "SVM")

    N   P   K  temperature   humidity        ph    rainfall label
0  90  42  43    20.879744  82.002744  6.502985  202.935536  rice
1  85  58  41    21.770462  80.319644  7.038096  226.655537  rice
2  60  55  44    23.004459  82.320763  7.840207  263.964248  rice
3  74  35  40    26.491096  80.158363  6.980401  242.864034  rice
4  78  42  42    20.130175  81.604873  7.628473  262.717340  rice
[[0.59285714 0.39285714 0.155      ... 0.76867044 0.55544287 0.64871193]
 [0.56428571 0.32857143 0.055      ... 0.63276254 0.50016844 0.27292564]
 [0.25714286 0.44285714 0.36       ... 0.06173618 0.5987286  0.21099654]
 ...
 [0.07857143 0.22142857 0.13       ... 0.43766892 0.48089445 0.28658863]
 [0.07857143 0.85       0.995      ... 0.76775145 0.46100935 0.18268542]
 [0.22857143 0.52142857 0.085      ... 0.56108125 0.57336975 0.11790781]]


[[0.72142857 0.08571429 0.21       ... 0.93886226 0.43037703 0.02033167]
 [0.7        0.02142857 0.23       ... 0.84309053 0.44329928 0.10358311]
 [0.42142857

/home/luisfuturist/GitHub/FIAP/activity-fiap-ai-p3a4/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/luisfuturist/GitHub/FIAP/activity-fiap-ai-p3a4/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/luisfuturist/GitHub/FIAP/activity-fiap-ai-p3a4/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

Cross-validated accuracy for Random Forest: 0.99 ± 0.01
Test set accuracy for Random Forest: 0.99
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        62
           1       1.00      1.00      1.00        57
           2       1.00      1.00      1.00        56
           3       1.00      1.00      1.00        68
           4       1.00      1.00      1.00        61
           5       1.00      1.00      1.00        60
           6       0.95      1.00      0.97        56
           7       1.00      1.00      1.00        59
           8       0.84      0.98      0.91        54
           9       1.00      1.00      1.00        71
          10       0.98      0.96      0.97        55
          11       1.00      0.95      0.97        60
          12       1.00      1.00      1.00        55
          13       0.97      0.99      0.98        70
          14       1.00      1.00      1.00        58
          15       1.00      1.00    

### 4.6 Evaluating Models

In [146]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load dataset
df = pd.read_csv('./data/dataset.csv')
print(df.head())

# Feature columns
X = df[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']]
# Label column
y = df['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=42, max_depth=7)
clf.fit(X_train, y_train) # Train the model
y_pred = clf.predict(X_test) # Make predictions

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print(classification_report(y_test, y_pred))

# Example of making a prediction for a new sample
new_sample = [[85, 40, 42, 21.0, 80.0, 6.4, 200.0]]  # Example input
predicted_label = clf.predict(new_sample)
print(f'Predicted label: {predicted_label[0]}')


    N   P   K  temperature   humidity        ph    rainfall label
0  90  42  43    20.879744  82.002744  6.502985  202.935536  rice
1  85  58  41    21.770462  80.319644  7.038096  226.655537  rice
2  60  55  44    23.004459  82.320763  7.840207  263.964248  rice
3  74  35  40    26.491096  80.158363  6.980401  242.864034  rice
4  78  42  42    20.130175  81.604873  7.628473  262.717340  rice
Accuracy: 0.81
              precision    recall  f1-score   support

       apple       1.00      1.00      1.00        48
      banana       1.00      1.00      1.00        36
   blackgram       0.70      0.95      0.80        37
    chickpea       1.00      1.00      1.00        47
     coconut       0.00      0.00      0.00        43
      coffee       1.00      1.00      1.00        39
      cotton       1.00      1.00      1.00        40
      grapes       1.00      1.00      1.00        36
        jute       1.00      0.21      0.35        43
 kidneybeans       1.00      1.00      1.00     

/home/luisfuturist/GitHub/FIAP/activity-fiap-ai-p3a4/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/luisfuturist/GitHub/FIAP/activity-fiap-ai-p3a4/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/luisfuturist/GitHub/FIAP/activity-fiap-ai-p3a4/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [149]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

# Load dataset
df = pd.read_csv('./data/dataset.csv')
print(df.head())

# Feature columns (adjust these according to your dataset's actual structure)
X = df[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']]
# Label column
y = df['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Feature scaling (optional but recommended for consistent results)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create a Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust the number of trees

# Train the model
rf.fit(X_train_scaled, y_train)

# Make predictions
y_pred = rf.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print(classification_report(y_test, y_pred))

# Example of making a prediction for a new sample
new_sample = [[85, 40, 42, 21.0, 80.0, 6.4, 200.0]]  # Example input
new_sample_scaled = scaler.transform(new_sample)  # Scale the new sample
predicted_label = rf.predict(new_sample_scaled)
print(f'Predicted label: {predicted_label[0]}')


    N   P   K  temperature   humidity        ph    rainfall label
0  90  42  43    20.879744  82.002744  6.502985  202.935536  rice
1  85  58  41    21.770462  80.319644  7.038096  226.655537  rice
2  60  55  44    23.004459  82.320763  7.840207  263.964248  rice
3  74  35  40    26.491096  80.158363  6.980401  242.864034  rice
4  78  42  42    20.130175  81.604873  7.628473  262.717340  rice
Accuracy: 0.99
              precision    recall  f1-score   support

       apple       1.00      1.00      1.00        48
      banana       1.00      1.00      1.00        36
   blackgram       1.00      1.00      1.00        37
    chickpea       1.00      1.00      1.00        47
     coconut       1.00      1.00      1.00        43
      coffee       1.00      1.00      1.00        39
      cotton       1.00      1.00      1.00        40
      grapes       1.00      1.00      1.00        36
        jute       0.89      0.98      0.93        43
 kidneybeans       1.00      1.00      1.00     

/home/luisfuturist/GitHub/FIAP/activity-fiap-ai-p3a4/.venv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [145]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Load dataset
df = pd.read_csv('./data/dataset.csv')
print(df.head())

# Feature columns
X = df[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']]
# Label column
y = df['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Create a Logistic Regression model
model = LogisticRegression(max_iter=200, multi_class='multinomial', solver='lbfgs')

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print(classification_report(y_test, y_pred))

# Example of making a prediction for a new sample
new_sample = [[85, 40, 42, 21.0, 80.0, 6.4, 200.0]]  # Example input
predicted_label = model.predict(new_sample)
print(f'Predicted label: {predicted_label[0]}')

    N   P   K  temperature   humidity        ph    rainfall label
0  90  42  43    20.879744  82.002744  6.502985  202.935536  rice
1  85  58  41    21.770462  80.319644  7.038096  226.655537  rice
2  60  55  44    23.004459  82.320763  7.840207  263.964248  rice
3  74  35  40    26.491096  80.158363  6.980401  242.864034  rice
4  78  42  42    20.130175  81.604873  7.628473  262.717340  rice


/home/luisfuturist/GitHub/FIAP/activity-fiap-ai-p3a4/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy: 0.95
              precision    recall  f1-score   support

       apple       1.00      1.00      1.00        48
      banana       0.97      1.00      0.99        36
   blackgram       0.74      0.92      0.82        37
    chickpea       1.00      1.00      1.00        47
     coconut       1.00      1.00      1.00        43
      coffee       0.97      1.00      0.99        39
      cotton       0.90      0.93      0.91        40
      grapes       1.00      1.00      1.00        36
        jute       0.88      0.86      0.87        43
 kidneybeans       0.98      0.98      0.98        45
      lentil       0.90      0.95      0.92        37
       maize       0.91      0.86      0.89        37
       mango       1.00      1.00      1.00        41
   mothbeans       0.94      0.70      0.80        47
    mungbean       0.93      1.00      0.96        37
   muskmelon       1.00      1.00      1.00        35
      orange       1.00      1.00      1.00        35
      papaya

/home/luisfuturist/GitHub/FIAP/activity-fiap-ai-p3a4/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/luisfuturist/GitHub/FIAP/activity-fiap-ai-p3a4/.venv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [144]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

# Load dataset
df = pd.read_csv('./data/dataset.csv')
print(df.head())

# Feature columns
X = df[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']]
# Label column
y = df['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create a KNN Classifier
k = 3  # You can experiment with different values of k
knn = KNeighborsClassifier(n_neighbors=k)

# Train the model
knn.fit(X_train_scaled, y_train)

# Make predictions
y_pred = knn.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print(classification_report(y_test, y_pred))

# Example of making a prediction for a new sample
new_sample = [[85, 40, 42, 21.0, 80.0, 6.4, 200.0]]  # Example input
new_sample_scaled = scaler.transform(new_sample)  # Scale the new sample
predicted_label = knn.predict(new_sample_scaled)
print(f'Predicted label: {predicted_label[0]}')


    N   P   K  temperature   humidity        ph    rainfall label
0  90  42  43    20.879744  82.002744  6.502985  202.935536  rice
1  85  58  41    21.770462  80.319644  7.038096  226.655537  rice
2  60  55  44    23.004459  82.320763  7.840207  263.964248  rice
3  74  35  40    26.491096  80.158363  6.980401  242.864034  rice
4  78  42  42    20.130175  81.604873  7.628473  262.717340  rice
Accuracy: 0.97
              precision    recall  f1-score   support

       apple       1.00      1.00      1.00        48
      banana       1.00      1.00      1.00        36
   blackgram       0.90      0.97      0.94        37
    chickpea       1.00      1.00      1.00        47
     coconut       0.93      1.00      0.97        43
      coffee       0.97      0.95      0.96        39
      cotton       0.95      1.00      0.98        40
      grapes       1.00      1.00      1.00        36
        jute       0.85      0.95      0.90        43
 kidneybeans       0.96      1.00      0.98     

/home/luisfuturist/GitHub/FIAP/activity-fiap-ai-p3a4/.venv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [143]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

# Load dataset
df = pd.read_csv('./data/dataset.csv')
print(df.head())

# Feature columns (assuming these are the correct feature names; adjust if necessary)
X = df[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']]
# Label column
y = df['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create an SVM Classifier
svm = SVC(kernel='poly')  # You can choose different kernels like 'linear', 'poly', 'sigmoid'

# Train the model
svm.fit(X_train_scaled, y_train)

# Make predictions
y_pred = svm.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print(classification_report(y_test, y_pred))

# Example of making a prediction for a new sample
new_sample = [[85, 40, 42, 21.0, 80.0, 6.4, 200.0]]  # Example input
new_sample_scaled = scaler.transform(new_sample)  # Scale the new sample
predicted_label = svm.predict(new_sample_scaled)
print(f'Predicted label: {predicted_label[0]}')


    N   P   K  temperature   humidity        ph    rainfall label
0  90  42  43    20.879744  82.002744  6.502985  202.935536  rice
1  85  58  41    21.770462  80.319644  7.038096  226.655537  rice
2  60  55  44    23.004459  82.320763  7.840207  263.964248  rice
3  74  35  40    26.491096  80.158363  6.980401  242.864034  rice
4  78  42  42    20.130175  81.604873  7.628473  262.717340  rice
Accuracy: 0.91
              precision    recall  f1-score   support

       apple       1.00      1.00      1.00        48
      banana       1.00      0.97      0.99        36
   blackgram       0.83      0.92      0.87        37
    chickpea       1.00      1.00      1.00        47
     coconut       1.00      0.98      0.99        43
      coffee       1.00      0.85      0.92        39
      cotton       1.00      0.88      0.93        40
      grapes       1.00      1.00      1.00        36
        jute       0.73      0.86      0.79        43
 kidneybeans       0.98      1.00      0.99     

/home/luisfuturist/GitHub/FIAP/activity-fiap-ai-p3a4/.venv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
